In [1]:
import sys
sys.path.append('./src/')
import numpy as np
import pickle
import torch
from global_var import *
from normalize import *
from data_load import *
from utils import *
from AE import AutoEncoder
from VAE import VAE
import KITree

torch.manual_seed(SEED)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# load data
"""
available datasets and subsets: 
cicids_custom: [Tuesday, Wednesday, Thursday, Friday]
toniot_custom: [
    backdoor, ddos, dos,
    injection, mitm, password,
    runsomware, scanning, xss
]
cse_improved: [server1, server2]
"""

dataset = 'toniot_custom'
subset = 'dos'
X_train, X_eval, y_train, y_eval = load_data(dataset, subset, mode='train')
X_test, y_test = load_data(dataset, subset, mode='test')
with open(os.path.join(NORMALIZER_DIR, f'{dataset}_{subset}.norm'), 'rb') as f:
    normalizer = pickle.load(f)
X_train, X_eval = normalizer.transform(X_train), normalizer.transform(X_eval)
X_test = normalizer.transform(X_test)

In [3]:
# load black box model
"""
available trained models: AE, VAE, OCSVM, IForest
"""

blackbox = 'AE'
try:
    model = torch.load(os.path.join(TARGET_MODEL_DIR, f'{blackbox}_{dataset}_{subset}.model'), map_location=DEVICE).to(DEVICE)
    thres = model.thres
except:
    with open(os.path.join(TARGET_MODEL_DIR, f'{blackbox}_{dataset}_{subset}.model'), 'rb') as f:
        model = pickle.load(f)
    thres = -model.offset_

In [4]:
# test well-trained blackbox model
y_origin = model.predict(X_test)
# sklearn models have different prediction format
if blackbox == 'OCSVM' or blackbox == 'IForest':
    y_origin[y_origin == 1] = 0
    y_origin[y_origin == -1] = 1
evaluate_predictions(y_test, y_origin)

TPR:  1.0
FPR:  0.018832391713747645
TNR:  0.9811676082862524
FNR:  0.0
Accuracy: 0.9869366427171783
Precision: 0.9591002044989775
Recall: 1.0
F1 Score: 0.9791231732776617


In [5]:
# prepare input of rule model
if blackbox == 'OCSVM' or blackbox == 'IForest':
    score = -model.score_samples(X_train)
    func = lambda x: -model.score_samples(x)
else:
    score = model.score_samples(X_train)
    func = lambda x: model.score_samples(x)

In [6]:
# extract rules from blackbox model using default hyperparameters
"""
here are some important hyperparameters you may try to calibrate:
max_levels = [5, 10, 15, 20, 25, 30]
n_beams = [2, 4, 6, 8, 10]
rhos = [0.01, 0.05, 0.1, 0.5, 1]
etas = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
"""
rule_model = KITree.KITree(func, thres)
rule_model.fit(X_train, score)

In [7]:
# predict() returns 0 if normal else 1
y_pred = rule_model.predict(X_test)
X_perturb = perturb_data_point(X_test)
y_perturb = rule_model.predict(X_perturb)
evaluate_rule_model(y_test, y_pred, y_origin, y_perturb)

Accuracy: 0.977139124755062
Precision: 0.9305555555555556
Recall: 1.0
F1 Score: 0.9640287769784173
Fidelity: 0.9902024820378837
robustness 1.0
TPR:  1.0
FPR:  0.03295668549905838
TNR:  0.9670433145009416
FNR:  0.0


In [7]:
"""
here are some useful functions:
get_depth(): get maximum depth of tree
get_leaf_num(): get leaf number of tree
"""
rule_model.get_depth(), rule_model.get_leaf_num()

(5, 21)

In [8]:
"""
get_rules_dict(feature name list, feature normalizer):
return a dict of rules where key is tree path to leaf and value is array of feature normality on this leaf;
[-inf, -inf] for all dimensions means always outputting abnormal on this leaf
"""
rule_model.get_rules_dict(CUSTOM_FEAT_COLS, normalizer)

{('dim 3 feat ps_mean <= 378.8089203433244',
  'dim 16 feat iat_max <= 0.9494180656981583',
  'dim 1 feat fwd_count <= 169.21179589013997',
  'dim 1 feat fwd_count <= 79.5294102875443',
  'dim 0 feat count <= 83.7232317671623'): array([[-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf],
        [-inf, -inf]]),
 ('dim 3 feat ps_mean <= 378.8089203433244',
  'dim 16 feat iat_max <= 0.9494180656981583',
  'dim 1 feat fwd_

In [9]:
"""
predict_rule_out(data sample, feature normalizer, feature name list=None):
return two lists; first is rules of tree path to a leaf;
second is rules of feature normality on this leaf, where "satisfy" indicates if data sample satisfy this rule;
second list is empty if this leaf is always determined abnormal
"""
rule_model.predict_rule_out(X_test[100], normalizer, CUSTOM_FEAT_COLS)

([{'feature_id': 3,
   'feature_name': 'ps_mean',
   'comparison': '<=',
   'value': 40.0,
   'threshold': 378.8089203433244},
  {'feature_id': 16,
   'feature_name': 'iat_max',
   'comparison': '<=',
   'value': 0.03418612480163574,
   'threshold': 0.9494180656981583},
  {'feature_id': 1,
   'feature_name': 'fwd_count',
   'comparison': '<=',
   'value': 100.0,
   'threshold': 169.21179589013997},
  {'feature_id': 1,
   'feature_name': 'fwd_count',
   'comparison': '>',
   'value': 100.0,
   'threshold': 79.5294102875443}],
 [{'feature_id': 0,
   'feature_name': 'count',
   'comparison': '<=',
   'value': 100.0,
   'threshold': 405.2808585327424,
   'satisfy': True},
  {'feature_id': 0,
   'feature_name': 'count',
   'comparison': '>',
   'value': 100.0,
   'threshold': 215.3123339338876,
   'satisfy': False},
  {'feature_id': 1,
   'feature_name': 'fwd_count',
   'comparison': '>',
   'value': 100.0,
   'threshold': 117.65902353189982,
   'satisfy': False},
  {'feature_id': 2,
   'fe